In [10]:
import os

import mlflow.sklearn
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize

In [35]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_id = client.create_experiment("Iris Classification Experiments")

In [57]:
class TreeModel:
    """
    DecisionTree classifier to predict trinary labels of flower dataset.
    """

    def __init__(self, **model_params):
        self._decision_tree = DecisionTreeClassifier(**model_params)
        self._params = model_params
        self.data = load_iris()
        
    @classmethod
    def create_instance(cls, **model_params):
        return cls(**model_params)
    
    @property
    def model(self):
        return self._decision_tree
    
    @property
    def params(self):
        return self._params
    
    def mlflow_run(self, run_name='Iris Classification Runs'):
        with mlflow.start_run(run_name=run_name) as run:
            run_id = run.info.run_uuid
            experiment_id = run.info.experiment_id
            
            X, y = self.data.data, self.data.target
            # y = label_binarize(y, classes=[0, 1, 2])
            
            X_train, X_test, y_train, y_test = \
                    train_test_split(X, y, test_size=0.3, random_state=3)
            
            self._decision_tree.fit(X_train, y_train)
            y_pred = self._decision_tree.predict(X_test)
            y_probs = self._decision_tree.predict_proba(X_test)
            
            mlflow.sklearn.log_model(self.model, "decision-tree-classifier")
            mlflow.log_params(self.params)
            
            acc = accuracy_score(y_test, y_pred))
            
            mlflow.log_metric("accuracy_score", acc)
            mlflow.log_metric("max_depth_of_trained_tree", self._decision_tree.tree_.max_depth)
            
            print("Accuracy Score =>", acc)
            print("Res max depth  =>", self._decision_tree.tree_.max_depth)
            return experiment_id, run_id

In [58]:
max_depths = [1, 2, 3]
for n in max_depths:
    params = {"max_depth": n, "random_state": 3}
    dtc = TreeModel.create_instance(**params)
    exp_id, run_id = dtc.mlflow_run()
    print(f"MLflow Run completed with run_id {exp_id} and experiment_id {run_id}")
    print("<->" * 40)

Accuracy Score => 0.6888888888888889
Precision      => 0.6888888888888889
Res max depth  => 1
MLflow Run completed with run_id 0 and experiment_id c6c6e7b6643b48de9bef60fa46accb6f
<-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><->
Accuracy Score => 0.9333333333333333
Precision      => 0.9333333333333333
Res max depth  => 2
MLflow Run completed with run_id 0 and experiment_id e2bc221f461b496fa3a0ff97c2c6f27f
<-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><->
Accuracy Score => 0.9333333333333333
Precision      => 0.9333333333333333
Res max depth  => 3
MLflow Run completed with run_id 0 and experiment_id 05112ce45dfa457da224b4ac93b0ea51
<-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><-><->
